In [1]:
#import gensim.utils as util
#util.test_local()

from gensim.models import Word2Vec
import logging
import os
from tqdm import tqdm
import sys
import json

from gensim import downloader
text8 = downloader.load('text8')

In [2]:
from flashtext import KeywordProcessor
keyword_processor = KeywordProcessor()
heword_processor = KeywordProcessor()

def load_json_pairs(input_file):
    with open(input_file, "r") as fp:
        pairs = json.load(fp)
    return pairs
PATH = '/home/manni/bias/counterfactual-data-substitution/cds/data/'
base_pairs = load_json_pairs(PATH+'cda_default_pairs_new.json')
#base_pairs = load_json_pairs('/home/manni/bias/counterfactual-data-substitution/cds/data/cda_default_filtered_pairs.json')
name_pairs = load_json_pairs(PATH+'names_pairs_1000_scaled.json')

exclusions = set()

for (male, female) in base_pairs:
    if male == 'he':
        print('continues')
        continue
    keyword_processor.add_keyword(male.lower(),male.lower()+'_'+female.lower())
    keyword_processor.add_keyword(female.lower(),male.lower()+'_'+female.lower())
    exclusions.add(male.lower())
    exclusions.add(female.lower())

for (male, female) in name_pairs:
    keyword_processor.add_keyword(male.lower(),male.lower()+'_'+female.lower())
    keyword_processor.add_keyword(female.lower(),male.lower()+'_'+female.lower())
    exclusions.add(male.lower())
    exclusions.add(female.lower())
heword_processor.add_keyword('he','she')
heword_processor.add_keyword('she','he')

continues


True

In [3]:
def get_window(lst, window):
    result = []
    last = 0
    
    for i, word in enumerate(lst):
        if word.lower() in {'he', 'she'}:
            window_start = max(last, i - window)
            window_end = min(len(lst), i + window + 1)
            seg = lst[window_start:window_end]
            result.extend(seg)
            last = window_end
    return result

In [4]:
# Set up logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
print('Cpu count:',os.cpu_count())

Cpu count: 48


In [5]:
def preprocess_text(text):
    he_she_text = get_window(text, 5)
    he_she_text = ' '.join(he_she_text)
    text = ' '.join(text)
    text = text.lower()
    he_she_text = he_she_text.lower()
    text_ = keyword_processor.replace_keywords(text) 
    heshe_ = heword_processor.replace_keywords(he_she_text) 
    return text_.split()+heshe_.split()


class SentenceIterator:
    def __iter__(self):
        for document in text8:
            _document = preprocess_text(document)
            yield _document

sentences = SentenceIterator()

In [6]:
# Initialize and train the Word2Vec model
model = Word2Vec(sentences=sentences, shrink_windows=True, sg=1, vector_size=300, window=5, min_count=3, workers=os.cpu_count())

2023-11-12 10:54:18,036 : INFO : collecting all words and their counts
2023-11-12 10:54:18,038 : DEBUG : {'uri': '/home/manni/gensim-data/text8/text8.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 10:54:18,094 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-12 10:54:54,903 : INFO : collected 254242 word types from a corpus of 17643378 raw words and 1701 sentences
2023-11-12 10:54:54,906 : INFO : Creating a fresh vocabulary
2023-11-12 10:54:55,298 : DEBUG : starting a new internal lifecycle event log for Word2Vec
2023-11-12 10:54:55,299 : INFO : Word2Vec lifecycle event {'msg': 'effective_min_count=3 retains 101640 unique words (39.98% of original 254242, drops 152602)', 'datetime': '2023-11-12T10:54:55.298609', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platfo

In [7]:
# Save the trained model
vocab = model.wv.key_to_index
emb_file = '/home/manni/embs/bw2v_merge_en_text8_mc3_epoch5_300.txt_2'
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vocab), 300)) 
    for word in tqdm(vocab, position=0):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in model.wv[word]])))
logging.info('Done')

2023-11-12 10:58:47,701 : INFO : Save trained word vectors
100%|██████████| 101640/101640 [00:20<00:00, 4930.38it/s]
2023-11-12 10:59:09,973 : INFO : Done


In [8]:
model.save('text8_bw2v.model')

2023-11-12 10:59:09,986 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'text8_bw2v.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-11-12T10:59:09.986844', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with-glibc2.35', 'event': 'saving'}
2023-11-12 10:59:09,990 : INFO : storing np array 'vectors' to text8_bw2v.model.wv.vectors.npy
2023-11-12 10:59:11,364 : INFO : storing np array 'syn1neg' to text8_bw2v.model.syn1neg.npy
2023-11-12 10:59:12,756 : INFO : not storing attribute cum_table
2023-11-12 10:59:12,757 : DEBUG : {'uri': 'text8_bw2v.model', 'mode': 'wb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 10:59:12,888 : INFO : saved text8_bw2v.model


In [9]:
from gensim.models import Word2Vec
model = Word2Vec.load("text8_bw2v.model")

2023-11-12 10:59:12,901 : INFO : loading Word2Vec object from text8_bw2v.model
2023-11-12 10:59:12,903 : DEBUG : {'uri': 'text8_bw2v.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 10:59:12,968 : INFO : loading wv recursively from text8_bw2v.model.wv.* with mmap=None
2023-11-12 10:59:12,969 : INFO : loading vectors from text8_bw2v.model.wv.vectors.npy with mmap=None
2023-11-12 10:59:13,039 : INFO : loading syn1neg from text8_bw2v.model.syn1neg.npy with mmap=None
2023-11-12 10:59:13,102 : INFO : setting ignored attribute cum_table to None
2023-11-12 10:59:13,855 : INFO : Word2Vec lifecycle event {'fname': 'text8_bw2v.model', 'datetime': '2023-11-12T10:59:13.855175', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with-glibc2.35', 'event': 'loaded'}


In [10]:
print('vocab before:{} pair vocab:{}'.format(len(model.wv.key_to_index),len(exclusions)))

vocab before:101640 pair vocab:2449


In [11]:
model.build_vocab(text8,update=True)

2023-11-12 10:59:13,888 : INFO : collecting all words and their counts
2023-11-12 10:59:13,889 : DEBUG : {'uri': '/home/manni/gensim-data/text8/text8.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 10:59:13,893 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-12 10:59:18,861 : INFO : collected 253854 word types from a corpus of 17005207 raw words and 1701 sentences
2023-11-12 10:59:18,865 : INFO : Updating model with new vocabulary
2023-11-12 10:59:19,378 : INFO : Word2Vec lifecycle event {'msg': 'added 1153 new unique words (0.45% of original 253854) and increased the count of 98885 pre-existing words (38.95% of original 253854)', 'datetime': '2023-11-12T10:59:19.378781', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with

In [12]:
print('vocab after:{} pair vocab:{}'.format(len(model.wv.key_to_index),len(exclusions)))

vocab after:102793 pair vocab:2449


In [13]:
oov = exclusions-set(model.wv.key_to_index)
print('OOV:{}'.format(len(oov)))

OOV:456


In [14]:
model.train(text8,total_examples=model.corpus_count,epochs=model.epochs)

2023-11-12 10:59:20,895 : WARNING : Effective 'alpha' higher than previous training cycles
2023-11-12 10:59:20,896 : INFO : Word2Vec lifecycle event {'msg': 'training model with 48 workers on 102793 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2023-11-12T10:59:20.896792', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with-glibc2.35', 'event': 'train'}
2023-11-12 10:59:20,913 : DEBUG : {'uri': '/home/manni/gensim-data/text8/text8.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 10:59:21,915 : INFO : EPOCH 0 - PROGRESS: at 5.76% examples, 724068 words/s, in_qsize 0, out_qsize 0
2023-11-12 10:59:22,931 : INFO : EPOCH 0 - PROGRESS: at 12.64% examples, 785874 words/s, in_qsize 0, out_qsize 1
202

(63081780, 85026035)

In [15]:
old_model = Word2Vec.load("text8_bw2v.model")

2023-11-12 11:00:39,583 : INFO : loading Word2Vec object from text8_bw2v.model
2023-11-12 11:00:39,585 : DEBUG : {'uri': 'text8_bw2v.model', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 11:00:39,624 : INFO : loading wv recursively from text8_bw2v.model.wv.* with mmap=None
2023-11-12 11:00:39,625 : INFO : loading vectors from text8_bw2v.model.wv.vectors.npy with mmap=None
2023-11-12 11:00:39,685 : INFO : loading syn1neg from text8_bw2v.model.syn1neg.npy with mmap=None
2023-11-12 11:00:39,743 : INFO : setting ignored attribute cum_table to None
2023-11-12 11:00:40,475 : INFO : Word2Vec lifecycle event {'fname': 'text8_bw2v.model', 'datetime': '2023-11-12T11:00:40.475473', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with-glibc2.35', 'event': 'loaded'}


In [16]:
old_model.build_vocab(text8,update=True)

2023-11-12 11:00:40,481 : INFO : collecting all words and their counts
2023-11-12 11:00:40,482 : DEBUG : {'uri': '/home/manni/gensim-data/text8/text8.gz', 'mode': 'rb', 'buffering': -1, 'encoding': None, 'errors': None, 'newline': None, 'closefd': True, 'opener': None, 'compression': 'infer_from_extension', 'transport_params': None}
2023-11-12 11:00:40,486 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-12 11:00:45,416 : INFO : collected 253854 word types from a corpus of 17005207 raw words and 1701 sentences
2023-11-12 11:00:45,419 : INFO : Updating model with new vocabulary
2023-11-12 11:00:45,938 : INFO : Word2Vec lifecycle event {'msg': 'added 1153 new unique words (0.45% of original 253854) and increased the count of 98885 pre-existing words (38.95% of original 253854)', 'datetime': '2023-11-12T11:00:45.938769', 'gensim': '4.3.2.dev0', 'python': '3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]', 'platform': 'Linux-5.15.0-76-generic-x86_64-with

In [17]:
new_vocab = old_model.wv.key_to_index 
for word in tqdm(exclusions):
    if word in new_vocab:
        old_model.wv[word]=model.wv[word]

100%|██████████| 2449/2449 [01:30<00:00, 27.17it/s]


In [18]:
emb_file = '/home/manni/embs/bw2v_MIX_en_text8_mc3_epoch5_300.txt_2'

In [19]:
# Save the trained model
logging.info('Save trained word vectors')
with open(emb_file, 'w', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(new_vocab), 300)) 
    for word in tqdm(new_vocab, position=0):
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in old_model.wv[word]])))
logging.info('Done')

2023-11-12 11:02:17,645 : INFO : Save trained word vectors
100%|██████████| 102793/102793 [00:20<00:00, 4981.70it/s]
2023-11-12 11:02:39,864 : INFO : Done


# de conflate copy

In [ ]:
# old model is now the mixed version
import numpy as np
DIM = old_model.vector_size
vectors = dict()
bvocab = old_model.wv.key_to_index
for word in tqdm(bvocab,leave=0):
    if word in exclusions:
        vec = old_model.wv.get_vector(word)
        vec = vec/np.linalg.norm(vec)
        vectors['@'+word]=vec
        continue
    words = word.split('_')
    if len(words)>1:
        for _word in words:
            if _word == 'he' or _word == 'she':
                continue
            vec = old_model.wv.get_vector(word)
            vec = vec/np.linalg.norm(vec)
            vectors[_word]=vec
    else:
        vec = old_model.wv.get_vector(word)
        vec = vec/np.linalg.norm(vec)
        vectors[word]=vec

In [ ]:
import gzip
emb_copy_file = emb_file+'.copy.gz'
with gzip.open(emb_copy_file, 'wt', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vectors), DIM))
    for word,vector in tqdm(vectors.items(), position=0):
        vector = vector/np.linalg.norm(vector)
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in vector])))

In [ ]:
emb_copy_file

# deconf avg 

In [ ]:
# old model is now the mixed version
import numpy as np
DIM = old_model.vector_size
vectors = dict()
bvocab = old_model.wv.key_to_index
for word in tqdm(bvocab,leave=0):
    words = word.split('_')
    if len(words)>1:
        for _word in words:
            if _word not in bvocab:
                continue
            vec_1 = old_model.wv.get_vector(word)
            vec_1 = vec/np.linalg.norm(vec_1)
            vec_2 = old_model.wv.get_vector(_word)
            vec_2 = vec/np.linalg.norm(vec_2)
            vec = vec_1 + vec_2
            assert vec.shape[0]==DIM
            vec = vec/np.linalg.norm(vec)
            vectors[_word]=vec
    else:
        vec = old_model.wv.get_vector(word)
        vec = vec/np.linalg.norm(vec)
        vectors[word]=vec

In [ ]:
import gzip
emb_avg_file = emb_file+'.avg.gz'
with gzip.open(emb_avg_file, 'wt', encoding='utf-8') as f:
    f.write('%d %d\n' % (len(vectors), DIM))
    for word,vector in tqdm(vectors.items(), position=0):
        vector = vector/np.linalg.norm(vector)
        f.write('%s %s\n' % (word, ' '.join([str(v) for v in vector])))

In [ ]:
emb_avg_file

# validation

In [ ]:
old_model.wv.most_similar('he')

In [ ]:
old_model.wv.most_similar('she')

In [ ]:
old_model.wv.most_similar('he_she')

In [ ]:
old_model.wv.most_similar('man_woman')

In [ ]:
old_model.wv.most_similar('bank')

In [ ]:
old_model.wv.most_similar('cat')

In [ ]:
for sent in sentences:
    print(len(sent))
    for i,word in enumerate(sent):
        if '_' in word:
            print(sent[i],sent[(len(sent)//2)+i])
            break
    input()

In [ ]:
model.wv.index2word(1)

In [ ]:
model.wv.similarity('he','she')

In [ ]:
model.wv.similarity('he','he_she')

In [ ]:
model.wv.similarity('she','he_she')